In [ ]:
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from skimage.transform import resize

In [ ]:
#HYPER PARAMETERS
confidence_threshold = 0.5
show_background = False

In [ ]:
def emptyFunction():
	pass

In [ ]:

# blackwindow having 3 color chanels
image = np.zeros((512, 512, 3), np.uint8)
image2 = np.zeros((512, 512, 3), np.uint8)
windowName ="LowerColor"
windowName2 ="UpperColor"

# window name
cv2.namedWindow(windowName)
cv2.namedWindow(windowName2)

# there trackbars which have the name
# of trackbars min and max value
cv2.createTrackbar('Blue1', windowName, 0, 255, emptyFunction)
cv2.createTrackbar('Green1', windowName, 0, 255, emptyFunction)
cv2.createTrackbar('Red1', windowName, 0, 255, emptyFunction)
cv2.createTrackbar('Blue2', windowName2, 0, 255, emptyFunction)
cv2.createTrackbar('Green2', windowName2, 0, 255, emptyFunction)
cv2.createTrackbar('Red2', windowName2, 0, 255, emptyFunction)

In [ ]:
def transformToNumber(number):
    for i in range(6,12):
        if str(number) == str(i):
            return str(i-6)

def transformToHandAndNumber(number):
    for i in range(6,12):
        if str(number) == str(i):
            return 'L'+ str(i-6)

    return 'D' + str(number)

#dodavanje kamere
camera = cv2.VideoCapture(0)

#Učitavanje modela
model = load_model('freshDatasetModel.h5')

frames = []
while(1):
    #čitanje videa po frameovima
    _, imageFrame = camera.read()

    #pretvaranje slike iz BGR(RGB color space) u HSV(hue-saturation-value)
    if cv2.waitKey(1) == 27:
	    break
    hsvFrame = cv2.cvtColor(imageFrame, cv2.COLOR_BGR2HSV)
    #Colorpicker
    cv2.imshow(windowName, image)
    cv2.imshow(windowName2, image2) 
    # values of blue, green, red
    blue1 = cv2.getTrackbarPos('Blue1', windowName)
    green1 = cv2.getTrackbarPos('Green1', windowName)
    red1 = cv2.getTrackbarPos('Red1', windowName)
    blue2 = cv2.getTrackbarPos('Blue2', windowName2)
    green2 = cv2.getTrackbarPos('Green2', windowName2)
    red2 = cv2.getTrackbarPos('Red2', windowName2)

    # merge all three color chanels and
    # make the image composites image from rgb
    image[:] = [blue1, green1, red1]
    image2[:] = [blue2, green2, red2]
    print(blue1, green1, red1)
    print(blue2, green2, red2)

    #POSTAVLJANJE RANGE-ova BOJA I DEFINIRANJE MASKI
    # white_lower = np.array([94, 80, 2], np.uint8)     DEFAULT
    # white_upper = np.array([120, 255, 255], np.uint8)
    white_lower = np.array([blue1, green1, red1], np.uint8)
    white_upper = np.array([blue2, green2, red2], np.uint8)
    white_mask = cv2.inRange(hsvFrame, white_lower, white_upper) 

    kernal = np.ones((5, 5), "uint8") 
    
    # For blue color 
    blue_mask = cv2.dilate(white_mask, kernal) 
    res_blue = cv2.bitwise_and(imageFrame, imageFrame,  
                            mask = blue_mask) 
    
    # Creating contour to track blue color 
    contours, hierarchy = cv2.findContours(blue_mask, 
                                        cv2.RETR_TREE, 
                                        cv2.CHAIN_APPROX_SIMPLE) 
    
    boxes = []

    for pic, contour in enumerate(contours): 
        area = cv2.contourArea(contour) 
        if(area > 300): 
            x, y, w, h = cv2.boundingRect(contour) 
            boxes.append((x, y, w, h))
            xmin = x
            xmax = xmin + w
            ymin = y
            ymax = ymin + h
            
            predicted = model.predict_proba(resize(white_mask[ymin:ymax, xmin:xmax], (64, 64),anti_aliasing=True).reshape(1,64,64,1))
    
            if(predicted.max() >= confidence_threshold):
                imageFrame = cv2.rectangle(imageFrame, (x, y),  
                                    (x + w, y + h),  
                                    (255, 0, 0), 2) 
                cv2.putText(imageFrame, transformToHandAndNumber((np.argmax(predicted))), (x, y), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1.0, 
                            (255, 0, 0))  
            elif show_background:
                imageFrame = cv2.rectangle(imageFrame, (x, y),  
                                    (x + w, y + h),  
                                    (255, 0, 0), 2) 
                cv2.putText(imageFrame, "background", (x, y), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1.0, 
                            (255, 0, 0))  
                            
    cv2.imshow("Multiple Color Detection in Real-TIme", imageFrame) 
    cv2.imshow("BlackNWhite", white_mask)

    # Program Termination 
    if cv2.waitKey(10) & 0xFF == ord('q'): 
        camera.release() 
        cv2.destroyAllWindows() 
        break